In [2]:
# import pandas as pd
# import ray
# from ray.data.datasource import FastFileMetadataProvider

In [1]:
import requests
from uberduck_ml_dev.vocoders.hifigan import AttrDict, Generator

In [2]:

# class Generator(torch.nn.Module):
#     __constants__ = ['lrelu_slope', 'num_kernels', 'num_upsamples', 'p_blur']
#     def __init__(self, h):
#         super(Generator, self).__init__()
#         self.num_kernels = len(h.resblock_kernel_sizes)
#         self.num_upsamples = len(h.upsample_rates)
#         self.conv_pre = weight_norm(Conv1d(80, h.upsample_initial_channel, 7, 1, padding=3))
#         self.p_blur = h.gaussian_blur['p_blurring']
#         self.gaussian_blur_fn = None
#         if self.p_blur > 0.0:
#             self.gaussian_blur_fn = GaussianBlurAugmentation(h.gaussian_blur['kernel_size'], h.gaussian_blur['sigmas'], self.p_blur)
#         else:
#             self.gaussian_blur_fn = nn.Identity()
#         self.lrelu_slope = LRELU_SLOPE

#         resblock = ResBlock1 if h.resblock == '1' else ResBlock2

#         self.ups = nn.ModuleList()
#         for i, (u, k) in enumerate(zip(h.upsample_rates, h.upsample_kernel_sizes)):
#             self.ups.append(weight_norm(
#                 ConvTranspose1d(h.upsample_initial_channel//(2**i), h.upsample_initial_channel//(2**(i+1)),
#                                 k, u, padding=(k-u)//2)))

#         self.resblocks = nn.ModuleList()
#         for i in range(len(self.ups)):
#             resblock_list = nn.ModuleList()
#             ch = h.upsample_initial_channel//(2**(i+1))
#             for j, (k, d) in enumerate(zip(h.resblock_kernel_sizes, h.resblock_dilation_sizes)):
#                 resblock_list.append(resblock(h, ch, k, d))
#             self.resblocks.append(resblock_list)

#         self.conv_post = weight_norm(Conv1d(ch, 1, 7, 1, padding=3))
#         self.ups.apply(init_weights)
#         self.conv_post.apply(init_weights)

#     def load_state_dict(self, state_dict):
#         new_state_dict = {}
#         for k, v in state_dict.items():
#             new_k = k
#             if 'resblocks' in k:
#                 parts = k.split(".")
#                 # only do this is the checkpoint type is older
#                 if len(parts) == 5:
#                     layer = int(parts[1])
#                     new_layer = f"{layer//3}.{layer%3}"
#                     new_k = f"resblocks.{new_layer}.{'.'.join(parts[2:])}"
#             new_state_dict[new_k] = v
#         super().load_state_dict(new_state_dict)

#     def forward(self, x):
#         if self.p_blur > 0.0:
#             x = self.gaussian_blur_fn(x)
#         x = self.conv_pre(x)
#         for upsample_layer, resblock_group in zip(self.ups, self.resblocks):
#             x = F.leaky_relu(x, self.lrelu_slope)
#             x = upsample_layer(x)
#             xs = torch.zeros(x.shape, dtype=x.dtype, device=x.device)
#             for resblock in resblock_group:
#                 xs += resblock(x)
#             x = xs / self.num_kernels
#         x = F.leaky_relu(x)
#         x = self.conv_post(x)
#         x = torch.tanh(x)

#         return x

#     def remove_weight_norm(self):
#         print('Removing weight norm...')
#         for l in self.ups:
#             remove_weight_norm(l)
#         for group in self.resblocks:
#             for block in group:
#                 block.remove_weight_norm()
#         remove_weight_norm(self.conv_pre)
#         remove_weight_norm(self.conv_post)

# class ResBlock1(torch.nn.Module):
#     __constants__ = ['lrelu_slope']
#     def __init__(self, h, channels, kernel_size=3, dilation=(1, 3, 5)):
#         super(ResBlock1, self).__init__()
#         self.h = h
#         self.lrelu_slope = LRELU_SLOPE
#         self.convs1 = nn.ModuleList([
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[0],
#                                padding=get_padding(kernel_size, dilation[0]))),
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[1],
#                                padding=get_padding(kernel_size, dilation[1]))),
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[2],
#                                padding=get_padding(kernel_size, dilation[2])))
#         ])
#         self.convs1.apply(init_weights)

#         self.convs2 = nn.ModuleList([
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
#                                padding=get_padding(kernel_size, 1))),
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
#                                padding=get_padding(kernel_size, 1))),
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=1,
#                                padding=get_padding(kernel_size, 1)))
#         ])
#         self.convs2.apply(init_weights)

#     def forward(self, x):
#         for c1, c2 in zip(self.convs1, self.convs2):
#             xt = F.leaky_relu(x, self.lrelu_slope)
#             xt = c1(xt)
#             xt = F.leaky_relu(xt, self.lrelu_slope)
#             xt = c2(xt)
#             x = xt + x
#         return x

#     def remove_weight_norm(self):
#         for l in self.convs1:
#             remove_weight_norm(l)
#         for l in self.convs2:
#             remove_weight_norm(l)


# class ResBlock2(torch.nn.Module):
#     __constants__ = ['lrelu_slope']
#     def __init__(self, h, channels, kernel_size=3, dilation=(1, 3)):
#         super(ResBlock2, self).__init__()
#         self.h = h
#         self.convs = nn.ModuleList([
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[0],
#                                padding=get_padding(kernel_size, dilation[0]))),
#             weight_norm(Conv1d(channels, channels, kernel_size, 1, dilation=dilation[1],
#                                padding=get_padding(kernel_size, dilation[1])))
#         ])
#         self.convs.apply(init_weights)
#         self.lrelu_slope = LRELU_SLOPE

#     def forward(self, x):
#         for c in self.convs:
#             xt = F.leaky_relu(x, self.lrelu_slope)
#             xt = c(xt)
#             x = xt + x
#         return x

#     def remove_weight_norm(self):
#         for l in self.convs:
#             remove_weight_norm(l)

            
# def get_padding(kernel_size, dilation=1):
#     return int((kernel_size*dilation - dilation)/2)

# def init_weights(m, mean=0.0, std=0.01):
#     classname = m.__class__.__name__
#     if classname.find("Conv") != -1:
#         m.weight.data.normal_(mean, std)


In [3]:
from torch.nn.utils import weight_norm
from torch.nn import Conv1d, ConvTranspose1d, AvgPool1d, Conv2d
import torch.nn as nn
LRELU_SLOPE = 0.1

In [4]:
HIFI_GAN_CONFIG_URL = "https://uberduck-models-us-west-2.s3.us-west-2.amazonaws.com/hifigan_22khz_config.json"
HIFI_GAN_GENERATOR_URL = "https://uberduck-models-us-west-2.s3.us-west-2.amazonaws.com/hifigan_libritts100360_generator0p5.pt"
from io import BytesIO
response = requests.get(HIFI_GAN_CONFIG_URL)
hifigan_config = response.json()

In [5]:
h = AttrDict(hifigan_config)
if 'gaussian_blur' in hifigan_config:
    hifigan_config['gaussian_blur']['p_blurring'] = 0.0
else:
    hifigan_config['gaussian_blur'] = {'p_blurring': 0.0}
    h['gaussian_blur'] = {'p_blurring': 0.0}

# state_dict_g = torch.load(vocoder_path, map_location='cpu')['generator']

# load hifigan
vocoder = Generator(h)
vocoder.load_state_dict(loaded['generator'])
# denoiser = Denoiser(vocoder)
# if to_cuda:
vocoder.cuda()
# denoiser.cuda()
vocoder.eval()

NameError: name 'loaded' is not defined

In [5]:
model = Generator(AttrDict(hifigan_config))

In [6]:
import torch
response = requests.get(HIFI_GAN_GENERATOR_URL, stream=True)
bio = BytesIO(response.content)
loaded = torch.load(bio)
model.load_state_dict(loaded['generator'])

<All keys matched successfully>

In [8]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_vocoder

get_vocoder()

Getting model config...
Loading pretrained model...
Got pretrained model...


Generator(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
  )
  (resblocks): ModuleList(
    (0): ModuleList(
      (0): ResBlock1(
        (convs1): ModuleList(
          (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
          (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
        )
        (convs2): ModuleList(
          (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          (2): 

In [3]:
# lj_df = pd.read_csv(
#     "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100.txt",
#     sep="|",
#     header=None,
#     quoting=3,
#     names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
# )

# paths = lj_df.path.tolist()
# transcripts = lj_df.transcript.tolist()
# speaker_ids = lj_df.speaker_id.tolist()

# parallelism_length = 400
# audio_ds = ray.data.read_binary_files(
#     paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )
# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )

# paths = ray.data.from_items(paths, parallelism=parallelism_length)
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# transcripts = ray.data.from_items(transcripts, parallelism=parallelism_length)
# transcripts_ds = transcripts.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# output_dataset = (
#     transcripts_ds.zip(audio_ds)
#     .zip(paths_ds)
#     .zip(speaker_ids_ds)
# )
# output_dataset = output_dataset.map_batches(
#     lambda table: table.rename(
#         columns={
#             "value_1": "transcript",
#             "value_2": "audio_bytes",
#             "value_3": "path",
#             "value_4": "speaker_id"
#         }
#     )
# )

In [1]:
# datum = ray_dataset.take(1)

4

In [1]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_ray_dataset, train_func, train_config, model_config
from ray.train.torch import TorchTrainer, TorchCheckpoint, TorchTrainer
from ray.air.config import ScalingConfig, RunConfig
from ray.tune import SyncConfig

class TorchCheckpointFixed(TorchCheckpoint):
    def __setstate__(self, state: dict):
        if "_data_dict" in state and state["_data_dict"]:
            state = state.copy()
            state["_data_dict"] = self._decode_data_dict(state["_data_dict"])
        super(TorchCheckpoint, self).__setstate__(state)

ray_dataset = get_ray_dataset()
train_config['n_group_size'] = model_config['n_group_size']
train_config['dur_model_config'] = model_config['dur_model_config']
train_config['f0_model_config'] = model_config['f0_model_config']
train_config['energy_model_config'] = model_config['energy_model_config']
train_config['v_model_config']=model_config['v_model_config']

trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config=train_config,
    scaling_config=ScalingConfig(
        num_workers=2, use_gpu=True, resources_per_worker=dict(CPU=4, GPU=1)
    ),
    run_config=RunConfig(
        sync_config=SyncConfig(upload_dir="s3://uberduck-anyscale-data/checkpoints")
    ),
    datasets={"train": ray_dataset},
)

# result = trainer.fit()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ray/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
2023-03-08 13:49:12,366	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.38.167:6379...
2023-03-08 13:49:12,372	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 


RuntimeEnvSetupError: Failed to set up runtime environment.
Failed to upload package /tmp/ray_latest_runtime_env.zip to the Ray cluster: Package size (893.72MiB) exceeds the maximum size of 500.00MiB. You can exclude large files using the 'excludes' option to the runtime_env or provide a remote URI of a zip file using protocols such as 's3://', 'https://' and so on, refer to https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#api-reference.

In [1]:
warmstart_checkpoint_path = "/home/ray/default/radtts++ljs-dap.pt"

In [2]:
from uberduck_ml_dev.exec.train_radtts_with_ray import model_config, ray_df_to_batch_radtts, get_ray_dataset
from uberduck_ml_dev.models.radtts import RADTTS
from ray.air import session

In [3]:
model = RADTTS(
    **model_config,
)

Applying spectral norm to text encoder LSTM
Applying spectral norm to context encoder LSTM


/home/ray/default/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered 

In [4]:
ray_dataset = get_ray_dataset()

2023-03-08 13:39:55,924	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.2.79:6379...
2023-03-08 13:39:55,935	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 


RuntimeEnvSetupError: Failed to set up runtime environment.
Failed to upload package /tmp/ray_latest_runtime_env.zip to the Ray cluster: Package size (893.69MiB) exceeds the maximum size of 500.00MiB. You can exclude large files using the 'excludes' option to the runtime_env or provide a remote URI of a zip file using protocols such as 's3://', 'https://' and so on, refer to https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#api-reference.

In [ ]:
dataset_shard = session.get_dataset_shard("train")

In [ ]:
for batch_idx, ray_batch_df = next(enumerate(
    dataset_shard.iter_batches(batch_size=batch_size)
))

In [ ]:
        batch_dict = ray_df_to_batch_radtts(batch)
        mel = to_gpu(batch_dict['mel'])
        speaker_ids = to_gpu(batch_dict['speaker_ids'])
        attn_prior = to_gpu(batch_dict['attn_prior'])
        f0 = to_gpu(batch_dict['f0'])
        voiced_mask = to_gpu(batch_dict['voiced_mask'])
        p_voiced = to_gpu(batch_dict['p_voiced'])
        text = to_gpu(batch_dict['text'])
        in_lens = to_gpu(batch_dict['input_lengths'])
        out_lens = to_gpu(batch_dict['output_lengths'])
        energy_avg = to_gpu(batch_dict['energy_avg'])


In [5]:
outputs = model(
            mel, speaker_ids, text, in_lens, out_lens,
            binarize_attention=binarize, attn_prior=attn_prior,
            f0=f0, energy_avg=energy_avg,
            voiced_mask=voiced_mask, p_voiced=p_voiced)

NameError: name 'mel' is not defined

In [3]:
from ray.air.integrations.wandb import  setup_wandb

In [4]:
help(setup_wandb) #entity

Help on function setup_wandb in module ray.air.integrations.wandb:

setup_wandb(config: Optional[Dict] = None, api_key: Optional[str] = None, api_key_file: Optional[str] = None, rank_zero_only: bool = True, **kwargs) -> Union[wandb.sdk.wandb_run.Run, wandb.sdk.lib.disabled.RunDisabled]
    Set up a Weights & Biases session.
    
    This function can be used to initialize a Weights & Biases session in a
    (distributed) training or tuning run.
    
    By default, the run ID is the trial ID, the run name is the trial name, and
    the run group is the experiment name. These settings can be overwritten by
    passing the respective arguments as ``kwargs``, which will be passed to
    ``wandb.init()``.
    
    In distributed training with Ray Train, only the zero-rank worker will initialize
    wandb. All other workers will return a disabled run object, so that logging is not
    duplicated in a distributed run. This can be disabled by passing
    ``rank_zero_only=False``, which will t

In [5]:
config = train_config
setup_wandb(config, project="radtts-ray", rank_zero_only=False)

2023-03-07 16:35:52,269	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_trial_id` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
2023-03-07 16:35:52,271	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_trial_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
2023-03-07 16:35:52,272	WARNING session.py:100 -- In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_experiment_name` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
wandb: Currently logged in as: uberduck. Use `wandb 

In [8]:
import torch

print("CUDA AVAILABLE: ", torch.cuda.is_available())
epochs = config["epochs"]
batch_size = config["batch_size"]
steps_per_sample = config["steps_per_sample"]
# gin_channels = config["gin_channels"]
sigma = config['sigma']
kl_loss_start_iter = config['kl_loss_start_iter']
binarization_start_iter = config['binarization_start_iter']


CUDA AVAILABLE:  True


NameError: name 'RADTTS' is not defined

In [1]:
from ray.air import session
session.get_world_size()

In neither tune session nor train session!
/home/ray/anaconda3/lib/python3.10/site-packages/ray/air/session.py:28: UserWarning: `get_world_size` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `1`.
  warnings.warn(


1

In [2]:
help(session.get_world_size)

Help on function get_world_size in module ray.air.session:

get_world_size() -> int
    Get the current world size (i.e. total number of workers) for this run.
    
    .. code-block:: python
    
        import time
        from ray.air import session
        from ray.air.config import ScalingConfig
    
        def train_loop_per_worker(config):
            assert session.get_world_size() == 4
    
        train_dataset = ray.data.from_items(
            [{"x": x, "y": x + 1} for x in range(32)])
        trainer = TensorflowTrainer(train_loop_per_worker,
            scaling_config=ScalingConfig(num_workers=1),
            datasets={"train": train_dataset})
        trainer.fit()
    
    **PublicAPI (beta):** This API is in beta and may change before becoming stable.



In [14]:
from uberduck_ml_dev.models.radtts import RADTTS

model = RADTTS(
    **model_config,
)
import ray.train as train
# model = train.torch.prepare_model(model, parallel_strategy_kwargs = dict(find_unused_parameters=True))
checkpoint_dict = None
global_step = 0
start_epoch = 0

optim = torch.optim.Adam(
    model.parameters(),
    lr = config["learning_rate"],
    weight_decay = config["weight_decay"]
)
scheduler = ExponentialLR(
    optim,
    config["weight_decay"],
    last_epoch=-1,
)
dataset_shard = session.get_dataset_shard("train")
global_step = 0
scaler = GradScaler()

criterion = RADTTSLoss(
    sigma,
    config['n_group_size'],
    config['dur_model_config'],
    config['f0_model_config'],
    config['energy_model_config'],
    vpred_model_config=config['v_model_config'],
    loss_weights=config['loss_weights']
)
attention_kl_loss = AttentionBinarizationLoss()
iteration = 0

Applying spectral norm to text encoder LSTM
Applying spectral norm to context encoder LSTM


NameError: name 'ExponentialLR' is not defined

(autoscaler +53m35s) Adding 1 node(s) of type worker-node-type-0.
(autoscaler +53m35s) Adding 1 node(s) of type worker-node-type-1.


(raylet, ip=10.0.3.123) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.3.123)   aiogrpc.init_grpc_aio()


(autoscaler +54m36s) Resized to 100 CPUs, 1 GPUs.


(raylet, ip=10.0.13.139) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.13.139)   aiogrpc.init_grpc_aio()


(autoscaler +54m50s) Resized to 104 CPUs, 2 GPUs.
(autoscaler +55m16s) Adding 1 node(s) of type worker-node-type-0.


(raylet, ip=10.0.20.186) /home/ray/anaconda3/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.20.186)   aiogrpc.init_grpc_aio()


(autoscaler +56m16s) Resized to 120 CPUs, 2 GPUs.


(raylet) [2023-03-07 17:31:47,223 E 369 369] (raylet) node_manager.cc:3040: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: c896a1f2d49a841de85d9f4ade62b71676469b02e6133224652550d8, IP: 10.0.37.211) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.0.37.211`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
[2023-03-07 17:32:41,086 E 1169933 1176438] core_worker.cc:569: :info_message: Attempting to recover 300 lost objects by re

In [ ]:


# discriminator = MultiPeriodDiscriminator(MODEL_CONFIG["use_spectral_norm"])
# discriminator = train.torch.prepare_model(discriminator)

checkpoint_dict = _load_checkpoint_dict()
if checkpoint_dict is None:
    global_step = 0
    start_epoch = 0
else:
    global_step = checkpoint_dict["global_step"]
    start_epoch = checkpoint_dict["epoch"]
    if session.get_world_size() > 1:
        model_sd = checkpoint_dict["model"]
        # NOTE(zach): Add audio embedding state dict if it is not present.
        # NOTE(zach): Pass strict=False due to different nuber of gin_channels
        model.load_state_dict(model_sd, strict=False)
    else:
        model_sd = _fix_state_dict(checkpoint_dict["model"])
        # NOTE(zach): Pass strict=False due to different nuber of gin_channels
        model.load_state_dict(model_sd, strict=False)
    del checkpoint_dict

# NOTE (Sam): replace with RAdam
optim = torch.optim.Adam(
    model.parameters(),
    lr = config["learning_rate"],
    weight_decay = config["weight_decay"]
)
scheduler = ExponentialLR(
    optim,
    config["weight_decay"],
    last_epoch=-1,
)
dataset_shard = session.get_dataset_shard("train")
global_step = 0
scaler = GradScaler()

criterion = RADTTSLoss(
    sigma,
    config['n_group_size'],
    config['dur_model_config'],
    config['f0_model_config'],
    config['energy_model_config'],
    vpred_model_config=config['v_model_config'],
    loss_weights=config['loss_weights']
)
attention_kl_loss = AttentionBinarizationLoss()
iteration = 0
for epoch in range(start_epoch, start_epoch + epochs):
    for batch_idx, ray_batch_df in enumerate(
        dataset_shard.iter_batches(batch_size=batch_size)
    ):
        torch.cuda.empty_cache()
        _train_step(
            ray_batch_df,
            model,
            optim,
            global_step,
            steps_per_sample,
            scaler,
            scheduler,
            criterion,
            attention_kl_loss,
            iteration,
            kl_loss_start_iter,
            binarization_start_iter,
        )
        global_step += 1
    if session.get_world_rank() == 0:
        # TODO(zach): Also save wandb artifact here.
        checkpoint = Checkpoint.from_dict(
            dict(
                epoch=epoch,
                global_step=global_step,
                model=model.state_dict(),
            )
        )
        session.report({}, checkpoint=checkpoint)
        artifact = wandb.Artifact(
            f"artifact_epoch{epoch}_step{global_step}", "model"
        )
        with tempfile.TemporaryDirectory() as tempdirname:
            checkpoint.to_directory(tempdirname)
            artifact.add_dir(tempdirname)
            wandb.log_artifact(artifact)
        iteration += 1

2023-03-07 16:19:08,608	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-07 16:19:08,633	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-07 16:19:08,653	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9.zip' (6.85MiB) to Ray cluster...
2023-03-07 16:19:08,744	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9.zip'.
(_get_read_tasks pid=5467, ip=10.0.46.245) 2023-03-07 16:19:10,287	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.


2023-03-07 16:19:11,683	WARNING trial_runner.py:1333 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (202 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(RayTrainWorker pid=235, ip=10.0.45.178) 2023-03-07 16:19:23,587	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:19:24,250	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:19:27,668	INFO bulk_e

(RayTrainWorker pid=417, ip=10.0.17.31) CUDA AVAILABLE:  True
(RayTrainWorker pid=417, ip=10.0.17.31) Applying spectral norm to text encoder LSTM
(RayTrainWorker pid=417, ip=10.0.17.31) Applying spectral norm to context encoder LSTM


(RayTrainWorker pid=417, ip=10.0.17.31) /tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/working_dir_files/_ray_pkg_ee2356972b78dbfda1fe20807a72a9e9/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
(RayTrainWorker pid=417, ip=10.0.17.31) The boolean parameter 'some' has been replaced with a string parameter 'mode'.
(RayTrainWorker pid=417, ip=10.0.17.31) Q, R = torch.qr(A, some)
(RayTrainWorker pid=417, ip=10.0.17.31) should be replaced with
(RayTrainWorker pid=417, ip=10.0.17.31) Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
(RayTrainWorker pid=417, ip=10.0.17.31)   W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor o

(RayTrainWorker pid=235, ip=10.0.45.178) CUDA AVAILABLE:  True
(RayTrainWorker pid=235, ip=10.0.45.178) Applying spectral norm to text encoder LSTM


...

(RayTrainWorker pid=235, ip=10.0.45.178) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=235, ip=10.0.45.178)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=417, ip=10.0.17.31)   result = 

(RayTrainWorker pid=417, ip=10.0.17.31) Loss: 5.8561625480651855
(RayTrainWorker pid=235, ip=10.0.45.178) Loss: 5.837704658508301


(RayTrainWorker pid=235, ip=10.0.45.178) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=235, ip=10.0.45.178)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=417, ip=10.0.17.31) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=417, ip=10.0.17.31)   result = 

(RayTrainWorker pid=235, ip=10.0.45.178) Loss: 5.733274459838867
(RayTrainWorker pid=417, ip=10.0.17.31) Loss: 5.9500956535339355


2023-03-07 16:23:53,988	WARNING tune.py:150 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-07 16:23:56,613	ERROR tune.py:821 -- Trials did not complete: [TorchTrainer_d9af1_00000]
2023-03-07 16:23:56,614	INFO tune.py:825 -- Total run time: 285.09 seconds (283.31 seconds for the tuning loop).
2023-03-07 16:23:56,614	WARNING tune.py:831 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
(TorchTrainer pid=5550, ip=10.0.46.245) 2023-03-07 16:23:56,616	INFO utils.py:57 -- Worker 1 has failed.


In [7]:
import os
os.environ[
        "TORCH_DISTRIBUTED_DEBUG"
    ] = "DETAIL"

In [8]:
result = trainer.fit()

(RayTrainWorker pid=303, ip=10.0.5.7) 2023-03-07 15:09:46,491	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:46,610	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:49,880	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> TaskPoolMapOperator[MapBatches(<lambda>)]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:55,809	WARNING plan.py:528 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for mor

(autoscaler +26s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +26s) Adding 5 node(s) of type worker-node-type-0.


(TorchTrainer pid=3738, ip=10.0.48.48) /home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/dataset_iterator.py:64: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset/DatasetPipeline as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(TorchTrainer pid=3738, ip=10.0.48.48)   warnings.warn(
(RayTrainWorker pid=1124519) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=303, ip=10.0.5.7) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=1124519) wandb: wandb version 0.13.11 is available!  To upgrade, please run:
(RayTrainWorker pid=1124519) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=1124519) wandb: Tracking run with wandb version 0.13.10
(RayTrainWorker pid=1124519) w

(RayTrainWorker pid=1124519) CUDA AVAILABLE:  True
(RayTrainWorker pid=1124519) Applying spectral norm to text encoder LSTM


(RayTrainWorker pid=1124519) /tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/working_dir_files/_ray_pkg_eccc6979c79037b7d167507f1405cae4/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
(RayTrainWorker pid=1124519) The boolean parameter 'some' has been replaced with a string parameter 'mode'.
(RayTrainWorker pid=1124519) Q, R = torch.qr(A, some)
(RayTrainWorker pid=1124519) should be replaced with
(RayTrainWorker pid=1124519) Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
(RayTrainWorker pid=1124519)   W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a

...

(RayTrainWorker pid=303, ip=10.0.5.7) Loss: 66.65056610107422
(RayTrainWorker pid=1124519) Loss: 66.45819854736328
(autoscaler +6m24s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +6m28s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m34s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m38s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m44s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m48s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m54s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m59s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m4s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m8s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m14s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m18s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m24s) Rem

(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:821] [Rank 1] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=123, OpType=BROADCAST, Timeout(ms)=1800000) ran for 1800693 milliseconds before timing out.
(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:456] Some NCCL operations have failed or timed out. Due to the asynchronous nature of CUDA kernels, subsequent GPU operations might run on corrupted/incomplete data.
(RayTrainWorker pid=1124519) [E ProcessGroupNCCL.cpp:461] To avoid data inconsistency, we are taking the entire process down.
(RayTrainWorker pid=1124519) [2023-03-07 15:42:17,362 E 1124519 1124595] logging.cc:97: Unhandled exception: St13runtime_error. what(): [Rank 1] Watchdog caught collective operation timeout: WorkNCCL(SeqNum=123, OpType=BROADCAST, Timeout(ms)=1800000) ran for 1800693 milliseconds before timing out.
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weig

Trial name,date,hostname,node_ip,pid,timestamp,trial_id
TorchTrainer_209d8_00000,2023-03-07_15-09-43,ip-10-0-48-48,10.0.48.48,3738,1678230583,209d8_00000


2023-03-07 15:42:30,245	ERROR tune.py:821 -- Trials did not complete: [TorchTrainer_209d8_00000]
2023-03-07 15:42:30,246	INFO tune.py:825 -- Total run time: 1974.69 seconds (1962.16 seconds for the tuning loop).


RayTaskError: [36mray::_Inner.train()[39m (pid=3738, ip=10.0.48.48, repr=TorchTrainer)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 360, in train
    raise skipped from exception_cause(skipped)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 336, in entrypoint
    return self._trainable_func(
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/base_trainer.py", line 737, in _trainable_func
    super()._trainable_func(self._merged_config, reporter, checkpoint_dir)
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 653, in _trainable_func
    output = fn()
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/base_trainer.py", line 647, in train_func
    trainer.training_loop()
  File "/home/ray/anaconda3/lib/python3.10/site-packages/ray/train/data_parallel_trainer.py", line 433, in training_loop
  

In [9]:
??librosa_mel_fn

Signature:
librosa_mel_fn(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm='slaney',
    dtype=<class 'numpy.float32'>,
)
Source:   
@cache(level=10)
def mel(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm="slaney",
    dtype=np.float32,
):
    """Create a Mel filter-bank.

    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.

    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal

    n_fft     : int > 0 [scalar]
        number of FFT components

    n_mels    : int > 0 [scalar]
        number of Mel bands to generate

    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)

    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``

    htk       : bool [scalar]
        use HTK formula instead of Slaney

    norm : {None, 'slaney', or num

In [1]:
from librosa.filters import mel as librosa_mel_fn
help(librosa_mel_fn)

Help on function mel in module librosa.filters:

mel(sr, n_fft, n_mels=128, fmin=0.0, fmax=None, htk=False, norm='slaney', dtype=<class 'numpy.float32'>)
    Create a Mel filter-bank.
    
    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.
    
    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal
    
    n_fft     : int > 0 [scalar]
        number of FFT components
    
    n_mels    : int > 0 [scalar]
        number of Mel bands to generate
    
    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)
    
    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``
    
    htk       : bool [scalar]
        use HTK formula instead of Slaney
    
    norm : {None, 'slaney', or number} [scalar]
        If 'slaney', divide the triangular mel weights by the width of the mel band
        (area normalization).
    
      

In [1]:
import librosa
from librosa.util import pad_center
librosa.__version__

'0.10.0'

In [2]:
# import pad_center
help(pad_center)

Help on function pad_center in module librosa.util.utils:

pad_center(data: 'np.ndarray', *, size: 'int', axis: 'int' = -1, **kwargs: 'Any') -> 'np.ndarray'
    Pad an array to a target length along a target axis.
    
    This differs from `np.pad` by centering the data prior to padding,
    analogous to `str.center`
    
    Examples
    --------
    >>> # Generate a vector
    >>> data = np.ones(5)
    >>> librosa.util.pad_center(data, size=10, mode='constant')
    array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.])
    
    >>> # Pad a matrix along its first dimension
    >>> data = np.ones((3, 5))
    >>> librosa.util.pad_center(data, size=7, axis=0)
    array([[ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.]])
    >>> # Or its second dimension
    >>> librosa.util.pad_cent

In [3]:
import numpy as np
pad_center(np.ones(1000), 5000)

TypeError: pad_center() takes 1 positional argument but 2 were given

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install librosa==0.8.1

  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.11.1 requires cython==0.29.28, but you have cython 0.29.33 which is incompatible.
tts 0.11.1 requires inflect==5.6.0, but you have inflect 6.0.2 which is incompatible.
tts 0.11.1 requires librosa==0.8.0, but you have librosa 0.8.1 which is incompatible.
tts 0.11.1 requires numpy==1.22.4; python_version == "3.10", but you have numpy 1.22.0 which is incompatible.


In [2]:
import pandas as pd
import ray
parallelism_length = 400

In [3]:
lj_df = pd.read_csv(
    "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100_edited.txt",
    sep="|",
    header=None,
    quoting=3,
    names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
)
speaker_ids = lj_df.speaker_id.tolist()
speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
speaker_ids_ds = speaker_ids_ds.map_batches(
    lambda x: x, batch_format="pyarrow", batch_size=None
)

2023-03-06 16:54:26,264	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-06 16:54:26,291	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-06 16:54:26,316	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip' (6.75MiB) to Ray cluster...
2023-03-06 16:54:26,438	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip'.


In [4]:
lj_df

,path,transcript,speaker_id
0,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"Printing, in the only sense with which we are ...",0
1,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,in being comparatively modern.,0
2,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,For although the Chinese took impressions from...,0
3,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"produced the block books, which were the immed...",0
4,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,the invention of movable metal letters in the ...,0
...,...,...,...
95,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,have now come into general use and are obvious...,0
96,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,a little reduced in ugliness. The design of th...,0
97,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"and the whole effect is a little too gray, owi...",0
98,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"It must be remembered, however, that most mode...",0


In [3]:
output_dataset = speaker_ids_ds.map_batches(
    lambda table: table.rename(
        columns={

            "value_1": "speaker_id"
        }
    )
)

In [4]:
output_dataset.speaker_id.tolist()

AttributeError: 'Dataset' object has no attribute 'speaker_id'

(autoscaler +2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +2s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +7s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +12s) Removing 2 nodes of type worker-node-type-0 (idle).


In [ ]:
pd.DataFrame(output_dataset)

In [ ]:
output_dataset

2023-03-06 16:50:22,749	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(<lambda>)]
MapBatches(<lambda>)->MapBatches(<lambda>): 100%|██████████| 100/100 [00:03<00:00, 28.88it/s]


In [39]:
#     lj_df = pd.read_csv(
#         "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/vctk_mic1/all_with_embs.txt",
#         sep="|",
#         header=None,
#         quoting=3,
#         names=["path", "speaker_id", "transcript", "dataset_audio_file_id", "emb_path"],
#     )

In [38]:
# lj_df['path'][0]

In [37]:
# transcripts_ds = ray.data.from_items(transcripts, parallelism=parallelism_length)
# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# dataset_audio_file_ids = ray.data.from_items(
#     dataset_audio_files, parallelism=parallelism_length
# )

# pitch_paths_ds = ray.data.read_binary_files(
#     pitch_paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )

# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# transcripts_ds = transcripts_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# dataset_audio_file_ids = dataset_audio_file_ids.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)
# pitches_paths_ds = pitch_paths_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
